# Take advantage of each block in try/except/else/finally

Pythonでの例外ハンドリングにおける実施したい動作は4つに大別できる。

それらはtry, except, else, finallyブロックで実施できる

これらはそれぞれ固有の使いかたがあり、異なる複数の使い方が役立つ(Item51)

## finally blocks

try/finallyの組み合わせは例外を投げたい時に使えるが、
例外が起きた時の後処理コードを走らせたい時にも使える。

よくあるのは、ファイルハンドラをきちんと閉じたい時。(Item43も参照)

In [4]:
handle = open('/tmp/random_data.txt')  # raise IOError
try:
    data = handle.read()  # raise UnicodeDecodeError
finally:
    # tryの後に必ず実行される
    handle.close() 

IOError: [Errno 2] No such file or directory: '/tmp/random_data.txt'

In [9]:
try:
    handle = open('/tmp/random_data.txt')  # raise IOError
    data = handle.read()  # raise UnicodeDecodeError
finally:
    # tryの後に必ず実行される
    handle.close() 

NameError: name 'handle' is not defined

readを使うとなんらかの例外が起こるが、
finallyブロックでhandleのcloseメソッドが保証される。

tryブロックの前にopenを呼ばないといけない
(openの時に発生する例外ではfinallyブロックは呼ばれて欲しくない)

## else blocks

try/except/elseの組み合わせは例外捕捉を分かりやすくする。

tryで例外が出ない時、elseブロックが実行される。
elseブロックはtryブロックのコード量を減らし、可読性を向上させる。

例えば、JSONのDictionaryデータを読み込み含まれるkeyを返したい時。

In [6]:
import json 
def load_json_key(data, key):
    try:
        result_dict = json.loads(data)
    except ValueError as e:
        rause KeyError from e
    else:
        return result_dict[key]

SyntaxError: invalid syntax (<ipython-input-6-4ab544558598>, line 6)

適切なJSONデータでない時、json.dataのデコードはValueErrorの例外を投げる。
例外はexceptブロックで捕捉されハンドルされる。

デコードが成功した場合、elseブロックのkey探索が実施される。
その際に例外が発生した場合、tryブロックの外のため、callerに伝わる。

elseはtry/exceptの後に続くことが見た目的に見分けることができ、
例外時の流れがわかりやすくなる。

## 全部入り



In [8]:
UNDEFINED = object()

def divide_json(path):
    handle = open(path, 'r+')  # raise IOError
    try:
        data = handle.read()   # raise UnicodeDecodeError
        op = json.loads(data)  # raise ValueError
        value = (
            op['numerator'] /
            op['denominator']) # raise ZeroDivisionError
    except ZeroDivisionError as e:
        return UNDEFINED
    else:
        op['result'] = value
        result = json.dumps(op)
        handle.seek(0)
        handle.write(result)  # raise IOError
        return value
    finally:
        # 常に実行される
        handle.close()

## Things to remember

* 例外が出るかにかかわらず、try/finally文ではコードを綺麗にしよう
* elseブロックはtryブロックのコード量を最小化するのに役立ち、見た目にも区分けできる
* elseブロックは成功したtryブロック後やfinallyブロックのクリーンアップ前の追加動作に使われる